In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import spacy

data = pd.read_csv("C:/Users/sharavana Kumar/Desktop/DiseaseAndSymptoms.csv")


In [13]:

data['text'] = data.apply(lambda row: ' '.join(row.dropna().values.tolist()[1:]), axis=1)



In [14]:
nlp = spacy.load("en_core_web_sm")
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

data['preprocessed_text'] = data['text'].apply(preprocess_text)


In [15]:
X = data['preprocessed_text']
y = data['Disease']

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X)

In [17]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_tfidf, y)

SVC(kernel='linear')

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [19]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

In [20]:
import joblib

joblib.dump(svm_classifier, 'model.pkl')
joblib.dump(tfidf_vectorizer, 'vector.pkl')

['vector.pkl']

In [ ]:
from fuzzywuzzy import process

def validate_symptoms(symptoms):

    dataset_symptoms = data.drop(columns=['Disease']).values.flatten().tolist()
    
    symptoms = symptoms.lower().split()
 
    matches = process.extractOne(' '.join(symptoms), dataset_symptoms)
  
    threshold = 80 
    
    if matches[1] >= threshold:
        return True
    else:
        return False
while True:
    input_symptoms = input("Enter symptoms (separated by spaces): ")
    
    if validate_symptoms(input_symptoms):
        break
    else:
        print("Invalid symptoms. Please enter valid symptoms.")

preprocessed_input = preprocess_text(input_symptoms)

input_vector = tfidf_vectorizer.transform([preprocessed_input])

predicted_label = svm_classifier.predict(input_vector)[0]

print("Predicted Label:", predicted_label)

In [ ]:

svm_classifier = joblib.load('model.pkl')
tfidf_vectorizer = joblib.load('vector.pkl')

test_data = pd.read_csv("C:/Users/sharavana Kumar/Desktop/DiseaseAndSymptoms.csv")

test_data['text'] = test_data.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

test_data['preprocessed_text'] = test_data['text'].apply(preprocess_text)

X_test = test_data['preprocessed_text']
y_test = test_data['Disease']

X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Make predictions on the test data
y_pred_test = svm_classifier.predict(X_test_tfidf)

# Calculate accuracy on test data
accuracy_test = accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", accuracy_test)
